In [3]:
import kagglehub
import os

path = kagglehub.dataset_download("rmisra/news-category-dataset")
print(path)

C:\Users\misse\.cache\kagglehub\datasets\rmisra\news-category-dataset\versions\3


In [4]:
import pandas as pd
import json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [5]:
file_path = "../data/News_Category_Dataset_v3.json"

with open(file_path, "r") as file:
    data = [json.loads(line.strip()) for line in file]

df = pd.DataFrame(data)
f = df[["headline", "category"]]

In [6]:
category_map = {
    "SCIENCE": "Science", "HEALTHY LIVING": "Science", "WELLNESS": "Science", 
    "ENVIRONMENT": "Science", "GREEN": "Science", "FOOD & DRINK": "Science", 
    "HEALTH": "Science", "POLITICS": "Politics", "WORLD NEWS": "Politics", "U.S. NEWS": "Politics", 
    "BUSINESS": "Politics", "CRIME": "Politics", "RELIGION": "Politics", "IMPACT": "Politics", 
    "BLACK VOICES": "Politics", "LATINO VOICES": "Politics", "ENTERTAINMENT": "Entertainment", 
    "COMEDY": "Entertainment", "STYLE & BEAUTY": "Entertainment", "CULTURE & ARTS": "Entertainment", 
    "MEDIA": "Entertainment", "ARTS & CULTURE": "Entertainment", "STYLE": "Entertainment", "ARTS": "Entertainment"
}
df["category"] = df["category"].map(category_map).fillna("Other")
print(df.head(n=15))

                                                 link  \
0   https://www.huffpost.com/entry/covid-boosters-...   
1   https://www.huffpost.com/entry/american-airlin...   
2   https://www.huffpost.com/entry/funniest-tweets...   
3   https://www.huffpost.com/entry/funniest-parent...   
4   https://www.huffpost.com/entry/amy-cooper-lose...   
5   https://www.huffpost.com/entry/belk-worker-fou...   
6   https://www.huffpost.com/entry/reporter-gets-a...   
7   https://www.huffpost.com/entry/puerto-rico-wat...   
8   https://www.huffpost.com/entry/mija-documentar...   
9   https://www.huffpost.com/entry/biden-un-russia...   
10  https://www.huffpost.com/entry/bc-soc-wcup-cap...   
11  https://www.huffpost.com/entry/man-sets-fire-p...   
12  https://www.huffpost.com/entry/fiona-threatens...   
13  https://www.huffpost.com/entry/twitch-streamer...   
14  https://www.huffpost.com/entry/virginia-thomas...   

                                             headline       category  \
0   Over 4 Mill

In [7]:
label_encoder = LabelEncoder()
df["category"] = label_encoder.fit_transform(df["category"])

In [8]:
tfidf = TfidfVectorizer(stop_words="english", max_features=5000)
X = tfidf.fit_transform(df["headline"])
y = df["category"]

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
model = LogisticRegression(max_iter=3000)
model.fit(X_train, y_train)


LogisticRegression(max_iter=3000)

In [11]:
y_pred = model.predict(X_test)

In [12]:
y_test_labels = label_encoder.inverse_transform(y_test)
y_pred_labels = label_encoder.inverse_transform(y_pred)
print(classification_report(y_test_labels, y_pred_labels))

# fuck it good enough

               precision    recall  f1-score   support

Entertainment       0.74      0.67      0.71      8257
        Other       0.66      0.72      0.69     13788
     Politics       0.74      0.74      0.74     12348
      Science       0.70      0.66      0.68      7513

     accuracy                           0.70     41906
    macro avg       0.71      0.70      0.70     41906
 weighted avg       0.71      0.70      0.70     41906



In [16]:
import joblib

joblib.dump(model, "news_category_model.pkl")
joblib.dump(tfidf, "vectorizer.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']

In [ ]:
def load_model_and_vectorizer():
    

    model = joblib.load("news_category_model.pkl")
    vectorizer = joblib.load("vectorizer.pkl")
    label_encoder = joblib.load("label_encoder.pkl")
    return model, vectorizer, label_encoder

def predict_categories(headlines):
    model, vectorizer, label_encoder = load_model_and_vectorizer()
    X_new = vectorizer.transform(headlines)
    predictions = model.predict(X_new)
    predicted_labels = label_encoder.inverse_transform(predictions)
    return predicted_labels

['Science' 'Politics']
